In [1]:
%load_ext autoreload
%autoreload 2

# Imports

In [2]:
import os
import pandas as pd

import numpy as np
import jax
jax.config.update('jax_platform_name', 'gpu')



In [3]:
fn_EcoCyc = '../data/sRNA/EcoCyc/EcoCyc_regulatory_RNAs.csv'
fn_RNAInter = os.path.join('..', 'data', 'sRNA', 'RNAInter', 'Download_data_RR.csv')
fn_srnatarbase = os.path.join('..', 'data', 'sRNA', 'sRNATarBase', 'sRNATarBase.csv')
ecocyc = pd.read_csv(fn_EcoCyc)
rnainter = pd.read_csv(fn_RNAInter)
srnatarbase = pd.read_csv(fn_srnatarbase)
# data = pd.concat([pd.read_csv(fn_RNAInter), pd.read_csv(fn_EcoCyc)])
try:
    rnainter = rnainter.drop(columns=['Unnamed: 0'])
    srnatarbase = srnatarbase.drop(columns=['Unnamed: 0'])
except:
    pass


### Start with RNAInter

In [4]:
np.sum(rnainter['Sequence1'].isna()) + np.sum(rnainter['Sequence2'].isna())

2

In [5]:
data = pd.DataFrame()
rnainter['Database'] = 'RNAInter'
for n, o in zip(['Name', 'ID', 'Category', 'Symbol', 'Sequence', 'Database'], ['Interactor1.Symbol', 'Raw_ID1', 'Category1', 'Interactor1.Symbol', 'Sequence1', 'Database']):
    data[n] = rnainter.drop_duplicates(subset=['Interactor1.Symbol'])[o]
data2 = pd.DataFrame()
for n, o in zip(['Name', 'ID', 'Category', 'Symbol', 'Sequence', 'Database'], ['Interactor2.Symbol', 'Raw_ID2', 'Category2', 'Interactor2.Symbol', 'Sequence2', 'Database']):
    data2[n] = rnainter.drop_duplicates(subset=['Interactor2.Symbol'])[o]

# data = pd.concat([data, rnainter.drop_duplicates(subset=['Interactor2.Symbol']).reset_index()[['Interactor2.Symbol', 'Raw_ID2', 'Category2', 'Sequence2']].rename(
#     columns={j: i for i, j in zip(['Name', 'ID', 'Category', 'Symbol', 'Sequence'], ['Interactor2.Symbol', 'Raw_ID2', 'Category2', 'Interactor2.Symbol', 'Sequence2'])})], axis=0)
data = pd.concat([data, data2])

data = data[~data['Name'].isna()]
data = data.drop_duplicates(subset=['Name'])

In [6]:
data['Symbol'].value_counts()


Symbol
acnA    1
phoP    1
sibC    1
sibB    1
sibA    1
       ..
luxS    1
malK    1
manX    1
manY    1
symR    1
Name: count, Length: 276, dtype: int64

### Now EcoCyc

In [7]:
ecocyc.head()

,Unnamed: 0,Ontology - all instances of class from My Favorites,Matches,Common-Name,GO-Terms,Names,Object ID,Gene,Sequence - DNA sequence,Copy-Number,Common-Name.1
0,0,6S-RNA,Regulatory-RNAs,6S RNA,NaN,6S RNA // SsrS // Ssr,6S-RNA,EG30099,ATTTCTCTGAGATGTTCGCAAGCGGGCCAGTCCCCTGAGCCGATAT...,NaN,ssrS
1,1,RDLD-RNA,Regulatory-RNAs,antisense regulatory RNA RdlD,GO:2000766 // GO:0030371,antisense regulatory RNA RdlD // RdlD // RdlD ...,RDLD-RNA,G0-9042,GTCTAGAGTCAAGATTAGCCCCCGTGGTGTTGTCAGGTGCATACCT...,NaN,rdlD
2,2,RNA0-441,Regulatory-RNAs,arfA-derived small regulatory RNA,GO:0006970 // GO:0003729,<i>arfA</i>-derived small regulatory RNA,RNA0-441,G0-17130,AAGAAAGTAAATCACTTTTTTACCACTGGTCTTCTGCTTTCAGGCG...,NaN,yrdG
3,3,RNA0-409,Regulatory-RNAs,e14 prophage; small regulatory RNA C0293,GO:0035194 // GO:0040033 // GO:0003729,e14 prophage; small regulatory RNA C0293 // C0293,RNA0-409,G0-8897,TACTCACCCGGGACTCGCCAGGGGACAGCCAACAGGCATTGGGTGC...,NaN,C0293
4,4,RNA0-162,Regulatory-RNAs,putative antisense regulatory RNA RdlB,GO:1902201,putative antisense regulatory RNA RdlB // RdlB,RNA0-162,G0-9606,GTCTGGTTTCAAGATTAGCCCCCGTTCTGTTGTCAGGTTTTACCTC...,NaN,rdlB


In [8]:
ecocyc['Category'] = 'sRNA'
ecocyc['Database'] = 'EcoCyc'
ecocyc = ecocyc.drop_duplicates(subset=['Common-Name'])

data = pd.concat([data, ecocyc.rename(columns={j: i for i, j in zip(['Name', 'ID', 'Symbol', 'Sequence'], ['Common-Name', 'Object ID', 'Common-Name.1', 'Sequence - DNA sequence'])})[
    ['Name', 'ID', 'Category', 'Symbol', 'Sequence', 'Database']]], axis=0)

### Finally sRNATarBase

In [9]:
srnatarbase

,sRNA,Target,Regulation,sRNA Binding Position,Target Binding Position,sRNA Sequence,Target Sequence
0,dsrA,hns,Repression,"['31..43', 'NA', '31..54']","['7..19', 'NA', '7..19; 401..411']",AACACATCAGATTTCCTGGTGTAACGAATTTTTTAAGTGCTTCTTG...,ATGAGCGAAGCACTTAAAATTCTGAACAACATCCGTACTCTTCGTG...
1,dsrA,rbsD,Repression,['NA'],['NA'],AACACATCAGATTTCCTGGTGTAACGAATTTTTTAAGTGCTTCTTG...,ATGAAAAAAGGCACCGTTCTTAATTCTGATATTTCATCGGTGATCT...
2,dsrA,argR,Repression,['NA'],['NA'],AACACATCAGATTTCCTGGTGTAACGAATTTTTTAAGTGCTTCTTG...,ATGCGAAGCTCGGCTAAGCAAGAAGAACTAGTTAAAGCATTTAAAG...
3,dsrA,ilvI,Repression,['NA'],['NA'],AACACATCAGATTTCCTGGTGTAACGAATTTTTTAAGTGCTTCTTG...,ATGGAGATGTTGTCTGGAGCCGAGATGGTCGTCCGATCGCTTATCG...
4,dsrA,rpoS,Induction,"['10..32', 'NA', 'NA']","['-119..-97', 'NA', 'NA']",AACACATCAGATTTCCTGGTGTAACGAATTTTTTAAGTGCTTCTTG...,ATGAGTCAGAATACGCTGAAAGTTCATGATTTAAATGAAGATGCGG...
...,...,...,...,...,...,...,...
406,ryhB,msrA,No Interaction,['NA'],['NA'],GCGATCAGGAAGACCCTCGCGGAGAACCTGAAAGCACGACATTGCT...,ATGAGTTTATTTGATAAAAAGCATCTGGTTTCCCCCGCCGATGCCC...
407,gcvB,hdeA,Induction,['NA'],['NA'],ACTTCCTGAGCCGGAACGAAAAGTTTTATCGGAATGCGTGTTCTGG...,ATGAAAAAAGTATTAGGCGTTATTCTTGGTGGTCTGCTTCTTCTGC...
408,gcvB,hdeB,Induction,['NA'],['NA'],ACTTCCTGAGCCGGAACGAAAAGTTTTATCGGAATGCGTGTTCTGG...,ATGAATATTTCATCTCTCCGTAAAGCGTTTATTTTTATGGGCGCTG...
409,Esr41,fliC,Induction,['NA'],['NA'],GATGCTCTAGGCATCACATTTTCTCCATGGGGTATTCCCTCCGCCG...,ATGGCACAAGTCATTAATACCAACAGCCTCTCGCTGATCACTCAAA...


# Delete duplicates responsibly

In [37]:
doubles = data['Symbol'].value_counts()[data['Symbol'].value_counts() > 1].index.to_list()

for d in doubles:
    
    if len(data[data['Symbol'] == d]['Sequence'].unique()) == 1:
        print('Removing duplicate of ', d, data[data['Symbol'] == d]['Name'].unique())
        data = data.drop(index=data[(data['Symbol'] == d) & (data['Database'] != 'EcoCyc')].index[0])
    else:
        print('Did not remove', d)
        # print(data[data['Symbol'] == d]['Sequence'].values)
    
data = data[~data['Sequence'].isna()]
data
# data[data['Symbol'] == d].index[0]

Did not remove istR
Did not remove ryhB
Did not remove sibB
Did not remove rprA
Did not remove dsrA
Did not remove sibD
Did not remove sibE
Did not remove rdlB
Did not remove rdlC
Did not remove rdlD
Did not remove oxyS
Did not remove omrB
Did not remove omrA
Did not remove mgrR
Did not remove micA
Did not remove micC
Did not remove micF
Did not remove fnrS
Did not remove symR
Did not remove ohsC
Did not remove rseX
Did not remove gadY
Did not remove gcvB
Did not remove rybB
Did not remove sibC
Did not remove dicF
Did not remove cyaR
Did not remove sdsR
Did not remove chiX
Did not remove arrS
Did not remove sokB
Did not remove mcaS
Did not remove sokC
Did not remove arcZ
Did not remove sgrS


,Name,ID,Category,Symbol,Sequence,Database
0,acnA,NCBI:946724,mRNA,acnA,ATTCGGAACGAGGCCTGAAGCAGTGTCGCCGTCCCTCTGCCTTGCA...,RNAInter
1,acrZ,NCBI:945365,mRNA,acrZ,ACTTACTACTGTCTTCGGGGGGTCCGAGGTTTCTGGGGGGTCGTAC...,RNAInter
2,arcZ,NCBI:2847690,sRNA,arcZ,CTCATGTTGACCGCTTGTTTAGCAGCTTCAAGGAAGCTGAAGGGCA...,RNAInter
4,argR,NCBI:947861,mRNA,argR,GACAATGGCGATAGTATGGCGGTTGTTTCTTTCCCATCTCTACTCA...,RNAInter
5,arrS,NCBI:11115378,sRNA,arrS,CTCTCCCTCTCTTTCTCTCTTCTCCGCGGCGATACGATCCGAGATG...,RNAInter
...,...,...,...,...,...,...
70,small regulatory RNA SroC,RNA0-124,sRNA,sroC,ACTAATTACAAGAACCAGGGGCGGAAATTCCAGCCCTCTCGATTGT...,EcoCyc
71,small regulatory RNA ZbiJ,RNA0-417,sRNA,zbiJ,TTTCCCTGTCTGTTTGCCGACAGACGCATATGCTCTAACCCTCATT...,EcoCyc
72,small RNA SibB,RYED-RNA,sRNA,sibB,GAGGGTAGAGCGGGGTTTCCCCCGCCCTGGTAGTCTTAGTAAGCGG...,EcoCyc
73,small RNA SibD,C0730-RNA,sRNA,sibD,ACAAGGGTGAGGGAGGATTTCTCCCCCCTCTGATTGGCTGTTAATA...,EcoCyc


In [38]:
data.to_csv('../data/sRNA/merged_EcoCyc_RNAInter.csv')